In [ ]:
# !pip install -U pandas numpy

In [ ]:
import torch
import bittensor as bt

from prompting.llm import HuggingFaceLLM
# from prompting.agent import HumanAgent
# from prompting.tasks import SummarizationTask
# from prompting.tools import WikiDataset

from transformers import pipeline


In [ ]:

import re
import torch
from transformers import Pipeline





class MockTokenizer(torch.nn.Module):

    def __init__(self):
        super(MockTokenizer, self).__init__()

        self.role_expr = '<|mock-{role}|>'

    def apply_chat_template(self, messages, **kwargs):
        prompt = ''
        for m in messages:
            role = self.role_expr.format(role=m['role'])
            content = m['content']
            prompt += f'<|mock-{role}|> {content}\n'

        return '\n'.join(prompt)


class MockModel(torch.nn.Module):

    def __init__(self, phrase):

        super(MockModel, self).__init__()

        self.tokenizer = MockTokenizer()
        self.phrase = phrase

    def __call__(self, messages):
        return self.forward(messages)

    def forward(self, messages):
        role_tag = self.tokenizer.role_expr.format(role='assistant')
        return f'{role_tag} {self.phrase}'



class MockPipeline():

    @property
    def tokenizer(self):
        return self.model.tokenizer

    def __init__(self, model_id='mock', device_map='mock-cuda', torch_dtype='torch.mock16', phrase='mock reply'):

        super(MockPipeline, self).__init__()

        self.model_id = model_id
        self.device_map = device_map
        self.torch_dtype = torch_dtype
        self.model = MockModel(phrase)

    def __repr__(self):
        return f'{self.__class__.__name__}(model_id={self.model_id}, device_map={self.device_map}, torch_dtype={self.torch_dtype})'

    def __call__(self, messages, **kwargs):
        return self.forward(messages, **kwargs)

    def forward(self, messages, **kwargs):

        output = self.model(messages)
        return self.postprocess(output)

    def postprocess(self, output, **kwargs):
        return [{'generated_text':output}]

    def preprocess(self, **kwargs):
        pass


In [ ]:
# bt.logging.info("🤖 Loading LLM model...")

# llm_pipeline = pipeline(
#     "text-generation",
#     model="HuggingFaceH4/zephyr-7b-beta",
#     torch_dtype=torch.bfloat16,
#     #device_map="cuda:0",
#     device_map="auto",
# )

# bt.logging.info("Creating task...")
# dataset = WikiDataset()
# context = dataset.next()

# task = SummarizationTask(llm_pipeline=llm_pipeline, context=context)

# bt.logging.info("Creating agent...")
# agent = HumanAgent(
#     task = task,
#     llm=llm_pipeline,
#     system_template=None,
#     begin_conversation=True
# )

In [ ]:
pipeline = MockPipeline(phrase='wut you say??')
pipeline

In [ ]:
agent = HuggingFaceLLM(pipeline=pipeline, system_prompt='u a badas')
agent.query('wassup')

In [1]:
import bittensor as bt

In [2]:
class MockMetagraph(bt.metagraph):

    def __init__(self, netuid=1, network='mock', sync=False):
        super(MockMetagraph, self).__init__(netuid=netuid, network=network, sync=sync)

        subtensor = bt.MockSubtensor()
        if not subtensor.subnet_exists(netuid):
            subtensor.create_subnet(netuid)

        for i in range(10):
            subtensor.force_register_neuron(netuid=netuid, hotkey=str(i), coldkey=str(i), balance=100000, stake=100000)

        self.sync(subtensor=subtensor)


m = MockMetagraph()

In [3]:
m.n

Parameter containing:
tensor(10)

In [4]:
m.axons

[AxonInfo( /ipv0/0.0.0.0:0, 0, 0, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 1, 1, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 2, 2, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 3, 3, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 4, 4, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 5, 5, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 6, 6, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 7, 7, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 8, 8, 0 ),
 AxonInfo( /ipv0/0.0.0.0:0, 9, 9, 0 )]

In [5]:
m.hotkeys

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
m.uids

Parameter containing:
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
m.last_update

Parameter containing:
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])